In [ ]:
import os
import ssl
import re
import json
import unittest
from pathlib import Path
from platform import python_version

import jupyterlab as jp
import nbdime
import nbgitpuller

def get_major_minor(version_str):
    return '.'.join(version_str.split('.')[:2])

def load_expected_versions() -> dict:
    with open('expected_versions.json', 'r') as f:
        return json.load(f)

def get_expected_version(dep_name: str) -> str:
    raw_value = expected_versions.get(dep_name)
    if raw_value is None:
        raise ValueError(f"Dependency {dep_name} not found in expected_versions.json")
    cleaned_version = re.sub(r'^\D+', '', raw_value)
    return get_major_minor(cleaned_version)

class TestPythonVersion(unittest.TestCase):
    def test_python_version(self):
        expected = get_expected_version("Python")
        actual = get_major_minor(python_version())
        self.assertEqual(actual, expected, f"Python version mismatch: expected {expected}, got {actual}")

class TestDependencyVersions(unittest.TestCase):
    def test_jupyterlab_version(self):
        expected = get_expected_version("JupyterLab")
        actual = get_major_minor(jp.__version__)
        self.assertEqual(actual, expected, f"JupyterLab version mismatch: expected {expected}, got {actual}")

    def test_nbgitpuller_version(self):
        expected = get_expected_version("nbgitpuller")
        actual = get_major_minor(nbgitpuller.__version__)
        self.assertEqual(actual, expected, f"nbgitpuller version mismatch: expected {expected}, got {actual}")

    def test_nbdime_version(self):
        expected = get_expected_version("nbdime")
        actual = get_major_minor(nbdime.__version__)
        self.assertEqual(actual, expected, f"nbdime version mismatch: expected {expected}, got {actual}")

class TestSecurity(unittest.TestCase):
    def test_jupyter_password_env(self):
        self.assertIn("JUPYTER_PASSWORD", os.environ, "Missing JUPYTER_PASSWORD env variable for login protection")

    def test_ssl_files_exist(self):
        cert_file = os.environ.get("JUPYTER_SSL_CERT", "/etc/jupyter/ssl/cert.pem")
        key_file = os.environ.get("JUPYTER_SSL_KEY", "/etc/jupyter/ssl/key.pem")
        self.assertTrue(os.path.exists(cert_file), f"SSL cert not found: {cert_file}")
        self.assertTrue(os.path.exists(key_file), f"SSL key not found: {key_file}")

    def test_ssl_certificate_validity(self):
        cert_file = os.environ.get("JUPYTER_SSL_CERT", "/etc/jupyter/ssl/cert.pem")
        try:
            context = ssl.create_default_context()
            context.load_cert_chain(certfile=cert_file)
        except Exception as e:
            self.fail(f"Invalid SSL cert: {e}")

    def test_host_not_public(self):
        host = os.environ.get("JUPYTER_HOST", "localhost")
        self.assertIn(host, ["localhost", "127.0.0.1"], f"Jupyter host is publicly exposed: {host}")

expected_versions = load_expected_versions()
unittest.main(argv=[''], verbosity=2, exit=False)